In [1]:
## Here kkr_imp_wc will be run from uuid: '23b13d9d-6e02-46c7-b3ac-17033b205b19'
## which is the converge kkr calculation for host Bi2Te3 TI.

In [2]:
#load aiida environment and connect to database
from aiida import load_profile

profile = load_profile()
print('profile name : ',profile.name)
print('profile database host name : ',profile.database_hostname)
print('default user : ',profile.default_user)

profile name :  default
profile database host name :  localhost
default user :  r.mozumder@fz-juelich.de


In [21]:
# load classes and functions
from aiida.orm import Code, Dict, load_node, StructureData
# from aiida.orm import
from aiida_kkr.tools import kkrparams
from aiida.engine import submit
from aiida_kkr.tools import kkrparams, plot_kkr
from aiida_kkr.workflows import kkr_flex_wc, kkr_imp_dos_wc, kkr_dos_wc

from aiida.orm import load_group

In [22]:
## Set the metadata options and code
metadata_option_1 = {'max_wallclock_seconds': 36000,'resources': 
               {'tot_num_mpiprocs': 48, 'num_machines': 1},
              'custom_scheduler_commands': 
              '#SBATCH --account=jara0191\n\nulimit -s unlimited; export OMP_STACKSIZE=2g',
              'withmpi': True
                    }
voro_code = Code.get_from_string('voro@claix18_init')
kkr_code = Code.get_from_string('kkr@claix18_init')
kkrimp_code = Code.get_from_string('kkrflex@claix18_init')

In [7]:
# Loading the converged kkr host calculation and the remote foder
Bi2Te3_conv_kkr = load_node('23b13d9d-6e02-46c7-b3ac-17033b205b19')
remote_last_calc_folder = Bi2Te3_conv_kkr.outputs.remote_folder


In [6]:
# Runing the host host_gf_writeout step

# setup the builder inputs
label = 'Fe:'+'Bi2Te3'
description = ('imp:d-block, repalced_atom:Bi,'+ 
               'Struc:uuid:c1626804-5b71-450c-91c4-aa7197d85279'+
               'kkr_conv:23b13d9d-6e02-46c7-b3ac-17033b205b19')

imp_info = {'Rcut': 4.0, 'Zimp': [26.0], 'ilayer_center': 4}

options, settings_imp, settings_vorostart = kkr_imp_wc.get_wf_defaults()
settings_imp['mag_init'] = True
settings_imp['retrieve_kkrflex'] = True
# settings_imp['convergence_criterion'] = 1e-8
settings_imp['nsteps'] = 100 
settings_imp['strmix'] = 0.02
settings_imp['threshold_aggressive_mixing'] = 0.05
# settings_imp['natom_in_cls_min'] = 35

builder_kkr_imp = kkr_imp_wc.get_builder()

builder_kkr_imp.options = Dict(dict=metadata_option_1)
builder_kkr_imp.metadata.label = label
builder_kkr_imp.metadata.description = description

builder_kkr_imp.impurity_info = Dict(dict=imp_info)
builder_kkr_imp.remote_data_host = remote_last_calc_folder
builder_kkr_imp.wf_parameters = Dict(dict=settings_imp)

builder_kkr_imp.voronoi = voro_code
builder_kkr_imp.kkr = kkr_code
builder_kkr_imp.kkrimp = kkrimp_code

# First kkr_imp is started here
# kkr_imp_start = submit(builder_kkr_imp)
# ilayer-3, pk:16100 uuid: 94c02cd3-57c8-4211-92bf-1478f2b93b68
# ilayer-4, pk:16637, uiid:'b146795e-3c0d-435d-924c-60834fc855a5'


Version of workflow: 0.8.1


'imp:d-block, repalced_atom:Bi,Struc:uuid:c1626804-5b71-450c-91c4-aa7197d85279kkr_conv:23b13d9d-6e02-46c7-b3ac-17033b205b19'

In [13]:
from aiida.engine import workfunction
from __future__ import print_function
from __future__ import absolute_import


In [ ]:
##### This is the edited part and my be final part
## By this function a long list of the single kkr_imp_wc could be run.
## Previous successful kkr_imp_wc process

from aiida.engine import workfunction
Fe_Bi2Te3_wc = load_node(16637)
host_gf = Fe_Bi2Te3_wc.outputs.remote_data_gf
imp_info_old = Fe_Bi2Te3_wc.inputs.impurity_info.get_dict()
options = Fe_Bi2Te3_wc.inputs.options

# @workfunction
def submition_bunch_wc(parent_wc, imp_num_each_wc=None, imp_Z_list=None,builder_options=None,
                       ilayers=[],label=None,description=None, imp_setup_dict_list=None,
                       group_label= None, group_description=None,
                       ):
    import time as t
    from aiida.common.exceptions import UniquenessError, NotExistent
    from aiida.common.constants import elements as elmt
    from aiida.orm import (CifData, Code, Dict, load_node,load_group, StructureData,
                          Group)

    """
params:
    parent_wc : (WorkchainNode, required= True): To get the impurity_info where the host
                atom will be repalced by the desired imp atom, rurun the WC.
                To get the host_gf 
    imp_num_each_wc : (int, required= False) : It represents impurity atom number palced in 
                     impurity info dict(). Either imp_num_each_wc or imp_setup_dict_list must
                     be given.
    imp_Z_list : (list, required= Flase): It contains list of all zimp for restarting the 
                 parent_imp_wc. Either imp_setup_dict_list or imp_Z_list have to be in param.
    builder_options : (dict or Dict, required = False): To setup the cluster options.
                      If not given parent_wc.option will be utilise
    ilayers: (list, required=False): This is the list of ilayers to run over the ilayer.
    

    """
## Todo: check group node, from successfull wc node, will be automatic created or not
## Todo: If not the try to add this feature
##

# To set the builder options
    if builder_options == None:
        try:
            options = parent_wc.inputs.options
        except NotExistent:
            msg = 'Neither builder_options is given nor it is found in parent_calculation'
            return msg
    else:
        options= builder_options
        print('Option is collected from params')

    # Collect required node from parent_wc
    imp_dict = parent_wc.inputs.impurity_info.get_dict()
    host_gf = parent_wc.outputs.remote_data_gf
    is_imp_dict_available = False
    
    # create group and store it
    try :
        existed_group= load_group(group_label)
        print('The given Group exists in DB.  So new successfull Nodes'
              + 'will be added with the group. Group pk {}'.format(existed_group.pk))
        created_group = existed_group
    except:
        if group_label == None:
            group_label='Restarted_wc_from_uuid_{}'.format(parent_wc.uuid)
        if group_description == None:
            group_description='The parent wc impurity info {}'.format(imp_dict)
        created_group=Group(label=group_label, description=group_description)
        created_group.store()

   
    ## To create all the possible imp_settings  
    all_possible_dict = {} ## All the posssible_dict combination
    all_succeful_dict = {} ## To collect all the successfull imp_dict
    all_imp_remaider = {} ## The leftover imp_dict without sucessfull imp_dict
    
    # Create dict of impurity_info dict either from imp_setup_dict_list 
    # or by combining imp_Z_list, ilayers 
    if imp_setup_dict_list==None:
        print("No impurity setup is provided. The impurity setup will be created as follows \n\
              Combination among all the atoms in the imp_Z_list consisting imp_num_each_wc\n\
              atom in each group. Each group will be set in the ilayers position. A \n\
              combination among the ilayers element will be consider, each group will \n\
              have the same length as numbe of atoms in each atom group" )
    
        if imp_num_each_wc == 1:
            ## order the dict
            dict_order = 0
            for Z in imp_Z_list[:] :
                    for i_layer in ilayers[:]:
                        dict_order += 1
                        imp_dict['Zimp'] = Z
#                         imp_dict['ilayer_center'] = i_layer
                        copy_imp_dict = imp_dict.copy()
                        all_possible_dict[dict_order] = copy_imp_dict

            all_imp_remaider = all_possible_dict.copy()
            is_imp_dict_available = True
            
    elif isinstance(imp_setup_dict_list, dict):
        all_possible_dict = imp_setup_dict_list ## All the posssible_dict combination
        all_imp_remaider = all_possible_dict.copy()
        is_imp_dict_available = True
        dict_order = len(imp_setup_dict_list)
        
    else :
        is_imp_dict_available = False
        return 'No impurity info is given by the imp_setup_dict_list nor imp_num_each_wc'
    
    ##  some pre assumed or pre known values 
    ####-------------------------
    # Change the workflow parameter here for higher
    kkr_imp_wc_dict = parent_wc.inputs.wf_parameters.get_dict()
    kkr_imp_wc_dict['nsteps'] = 300
    kkr_imp_wc_dict['convergence_criterion'] = 1e-07
    kkr_imp_wc_dict['retrieve_kkrflex'] = False
    
    ####-------------------------
    if is_imp_dict_available :
        # To submit max number of wc at one time
        max_wc_submition = 8
        stop_here = False
        bin_success_info = {}
        bin_= {}   ## Use it as submitted imp info
        ## wc counting info
        builder_iter = 0 ## for tracking the submitted wc
        finished_iter = 0 ## tracking the finished wc
        total_wc = dict_order ## Total number of wc to be submitted
        
        # 
        if max_wc_submition > total_wc:
            max_wc_submition = total_wc
        created_group_node_list = list(created_group.nodes)
        group_node_label_list = [node.label for node in created_group_node_list[:]]
        ## To run all the calcualtion according to the all_possible_dict
        for order, imp_settings in all_possible_dict.items():
            ## stop_here stop all the sumbmition function if exit_status != 0
            if stop_here==True:
                break
                print('At least work chain is finished with '+
                      'exit_code not 0, and stop here break 000')

    ####---------------------------------------------------
            zatom = imp_settings['Zimp']
            imp_symbol = elmt[zatom]['symbol']
            wc_label = imp_symbol + ':Bi2Te3'
            print(wc_label)
          
            new_imp_dict = imp_settings.copy()
            ## step for sumitting the one job
            builder = parent_wc.get_builder_restart()
            # To remove 'remote_data_host' from builder dict
            builder.pop('remote_data_host')
            builder.remote_data_gf = host_gf
            builder.wf_parameters = Dict(dict=kkr_imp_wc_dict)
            builder.impurity_info = Dict(dict=new_imp_dict)
            builder.metadata.label = str(wc_label)
            builder.options = options
            builder_iter += 1
            submition = submit(builder)
            t.sleep(30)
            ## Here the bin ony takes upto eight elements of group
            bin_element = {'pk': submition.pk,
                                  'uuid': submition.uuid,
                                  'exit_status': None,
                                  'track_num': order, 'imp_info' :new_imp_dict}
            bin_[order] = bin_element
            ## The keep the activated wc equal or below the max_wc_submition
            ## that is the maximum work-chain to be in running 
            while ((builder_iter-finished_iter >= max_wc_submition) or (total_wc == builder_iter) or (stop_here==True) ):

                t.sleep(120)
#                 print('sleeping for 120 second starting of the while loop')
                bin_success_wc = []
                bin_failure_wc = []
    
                if (len(bin_) == 0) :
                    ## Break the while loop here
                    break
                    
                for calc_order, sub_info in bin_.items():
                    wc_pk = bin_[calc_order]['pk']
                    wc_node = load_node(wc_pk)
                    if wc_node.is_finished == True:
                        if wc_node.exit_status == 0:
                            print('one clalc is finished')
                            bin_[calc_order]['exit_status']=0

                            ## To add the dict as a successfull dict
                            copy_to_success = all_possible_dict[calc_order].copy()
                            all_succeful_dict[calc_order] = (copy_to_success)

                            ## To delete the dict from the all_imp_remainder as it is the copy of all_possible_dict
                            all_imp_remaider.pop(calc_order)

                            ## To collect the successfull calc info such as pk, uuid
                            copy_sucess_info = bin_[calc_order].copy()
                           # before storing the node check is node is available there
                            if wc_node.label in group_node_label_list[:]:
                                print('The wc_node pk : {} and label {} is already exist on the group {}'.format(wc_node.pk, wc_node.label, created_group.label))
                                created_group.add_nodes(wc_node)
                            else:
                                created_group.add_nodes(wc_node)
                                
                            bin_success_info[calc_order] = copy_sucess_info
                            ## To reduce the dict from the bin_ as it is boundary that can be submit wc at one time
                            bin_success_wc.append(calc_order)
                            finished_iter += 1
                        else :
                            bin_failure_wc.append(calc_order)
                            stop_here = True
                            print('---exit_status is not 0. So, one inconistency in wc {} is found'.format(wc_pk))

                ## To delete the element from bin_ as this wc are finished 
                ## and create the new space for others
                if (bin_success_wc != []):
                    for i in bin_success_wc[:]:
                        bin_.pop(i)
                        
                if (bin_failure_wc != []):
                    for i in bin_failure_wc[:]:
                        bin_.pop(i)
                
    return all_possible_dict, all_succeful_dict, all_imp_remaider, bin_success_info


imp_num_each_wc = 1
i_layers = [4]
d_elmt_zatom = [*range(21,31, 1)] + [*range(39,49, 1)]
builder_options = Dict(dict=metadata_option_1)

group_label='3d_4d_dopants_Bi2Te3_ilayer4'


# all_possible_dict, all_succeful_dict, all_imp_remaider, bin_success_info  = submition_bunch_wc(parent_wc= Fe_Bi2Te3_wc, imp_num_each_wc= 1,
#                                                                                                imp_Z_list= d_elmt_zatom,builder_options= builder_options,
#                                                                                                ilayers= i_layers, group_label=group_label)
# finished_bin

# all_possible_dict, all_succeful_dict, all_imp_remaider, bin_success_info  = submition_bunch_wc(Fe_Bi2Te3_wc, builder_option, imp_setup_dict_list=imp_setup_dict_list,
#                                                                                               group_label='3d_4d_dopants_Bi2Te3_ilayer4')


Option is collected from params
The given Group exists in DB.  So new successfull Nodeswill be added with the group. Group pk 91
No impurity setup is provided. The impurity setup will be created as follows 
              Combination among all the atoms in the imp_Z_list consisting imp_num_each_wc
              atom in each group. Each group will be set in the ilayers position. A 
              combination among the ilayers element will be consider, each group will 
              have the same length as numbe of atoms in each atom group
Sc:Bi2Te3
Ti:Bi2Te3
V:Bi2Te3
Cr:Bi2Te3
Mn:Bi2Te3
Fe:Bi2Te3
Co:Bi2Te3
Ni:Bi2Te3
one clalc is finished
Cu:Bi2Te3
one clalc is finished
Zn:Bi2Te3
one clalc is finished
Y:Bi2Te3
one clalc is finished
Zr:Bi2Te3
one clalc is finished
Nb:Bi2Te3
one clalc is finished
Mo:Bi2Te3
one clalc is finished
Tc:Bi2Te3
one clalc is finished
Ru:Bi2Te3
one clalc is finished
Rh:Bi2Te3
one clalc is finished
Pd:Bi2Te3
one clalc is finished
Ag:Bi2Te3
one clalc is finished
Cd:Bi2

In [16]:
d_elmt_zatom = [*range(21,31, 1)] + [*range(39,49, 1)]


In [17]:
## This Part is intended to run the kkr_imp_dos_wc from the previous kkr_imp_wc
# Inputs:
# Features:

# Trying to run single imp DOS calculation

parent_kkrimp = load_node(16195)

## Preparation the inputs
dos_wf_parameters = {'EMIN': -5,
                     'EMAX': 10}

code = Code()
kkr_code = code.get_from_string('kkr@claix18_init')
kkrimp_code = code.get_from_string('kkrflex@claix18_init')
options = metadata_option_1
imp_info = parent_kkrimp.inputs.impurity_info
kkrimp_sfd = parent_kkrimp.outputs.converged_potential

# Now to submit the kkr_dos_wc node
kkrdoswc = kkr_imp_dos_wc.get_builder()

kkrdoswc.kkr = kkr_code
kkrdoswc.kkrimp = kkrimp_code
kkrdoswc.impurity_info = imp_info
kkrdoswc.imp_pot_sfd = kkrimp_sfd
kkrdoswc.options = Dict(dict=options)
kkrdoswc.wf_parameters = Dict(dict=dos_wf_parameters)
kkrdoswc_activate = submit(kkrdoswc)

In [30]:
## 
def dos_wc_branch(parent_kkrimp_wc_group, parent_kkrimp_wc_list=[]):
    ## Preparation the inputs
    dos_wf_parameters = {'EMIN': -5,
                         'EMAX': 10}
    parent_kkrimp_wc_group = load_group(parent_kkrimp_wc_group)
    wc_list = list(parent_kkrimp_wc_group.nodes)
    for node in wc_list[:]:
        print(node.pk)
        print(node.label)
    for node in wc_list[1:]:
        parent_kkrimp = load_node(node.pk)

        code = Code()
        kkr_code = code.get_from_string('kkr@claix18_init')
        kkrimp_code = code.get_from_string('kkrflex@claix18_init')
        options = metadata_option_1
        imp_info = parent_kkrimp.inputs.impurity_info
        kkrimp_sfd = parent_kkrimp.outputs.converged_potential
        label = parent_kkrimp.label + 'dos'
        # Now to submit the kkr_dos_wc node
        kkrdoswc = kkr_imp_dos_wc.get_builder()

        kkrdoswc.kkr = kkr_code
        kkrdoswc.kkrimp = kkrimp_code
        kkrdoswc.impurity_info = imp_info
        kkrdoswc.imp_pot_sfd = kkrimp_sfd
        kkrdoswc.options = Dict(dict=options)
        kkrdoswc.wf_parameters = Dict(dict=dos_wf_parameters)
        kkrdoswc.metadata.label = label
#         kkrdoswc_activate = submit(kkrdoswc)
    
dos_wc_branch(91)
    

18833
Sc:Bi2Te3
18853
Ti:Bi2Te3
18875
V:Bi2Te3
18896
Cr:Bi2Te3
18917
Mn:Bi2Te3
18938
Fe:Bi2Te3
19012
Ni:Bi2Te3
19180
Cu:Bi2Te3
19226
Zn:Bi2Te3
19257
Y:Bi2Te3
19303
Zr:Bi2Te3
19364
Nb:Bi2Te3
19410
Mo:Bi2Te3
19456
Tc:Bi2Te3
19551
Ru:Bi2Te3
19597
Rh:Bi2Te3
19642
Pd:Bi2Te3
19689
Ag:Bi2Te3
19720
Cd:Bi2Te3
